In [1]:
import sys
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
import MDAnalysis as mda
import torch
import torch.nn as nn
import torch.nn.functional as F
from primitives import gaussian_blur, get_box_atm_indices, Residue
from scipy.ndimage import gaussian_filter
from scipy.signal.windows import gaussian
from matplotlib.pyplot import figure
# from pyuul import VolumeMaker # the main PyUUL module
from pyuul import utils
from myvolmaker import Voxels
from proteinshake.tasks import EnzymeClassTask, ProteinProteinInterfaceTask
from proteinshake.datasets import Dataset
from torch.utils.data import DataLoader

# Use proteins with Enzyme Class annotations
# Convert them to voxels with a voxelsize of 10 Angstrom
# Load into PyTorch data structures
# task = ProteinProteinInterfaceTask().to_voxel(voxelsize=10).torch()
# task = ProteinProteinInterfaceTask()
# task.to_voxel(voxelsize=10).torch()
# train, test = DataLoader(task.train), DataLoader(task.test)

casa = Path("/home/pbarletta/labo/23/paco")
dlp = Path("/home/pbarletta/labo/22/locuaz/rebin/dlpacker")
weights_path = Path(dlp, "DLPacker_weights.h5")
lib_path = Path(dlp, "library.npz")
charges_path = Path(dlp, "charges.rtp")

In [2]:
input_pdb_fn = Path(casa, "d11.pdb")
coords, atname = utils.parsePDB(input_pdb_fn)
atoms_channel = utils.atomlistToChannels(atname)
radius = utils.atomlistToRadius(atname)
resolution = 1

volmaker = Voxels(device="cpu",sparse=False)
voxelized_volume = volmaker(coords, radius, atoms_channel,
                            resolution=resolution, cubes_around_atoms_dim=5,
                            function="gaussian")
voxels = voxelized_volume.sum(1)[:, None, :, :]
sp_voxels = voxels.to_sparse()
sp_voxels = sp_voxels[0, 0].coalesce()
volmaker.boxsize, volmaker.lato

((69, 98, 89), 11)

In [3]:
coords = volmaker.get_voxels_coordinates(atoms_channel)

In [4]:
def write_pdb_dots(voxel_coords, voxel_density, ndots: int, out_fn) :
    u = mda.Universe.empty(n_atoms = ndots, trajectory=True)
    u.add_TopologyAttr('name', ['H'] * ndots)
    u.add_TopologyAttr('type', ['H'] * ndots)
    u.add_TopologyAttr('tempfactor', np.array(voxel_density))
    u.atoms.positions = np.array(voxel_coords) 
    u.atoms.write(out_fn)

In [5]:
nvoxels = torch.tensor(coords.shape[:-1]).prod().item()
flat_coords = coords.view((nvoxels, 3))

write_pdb_dots(flat_coords,
               voxels.view(flat_coords.shape[0]), flat_coords.shape[0], "../cubo.pdb")

/home/pbarletta/mambaforge/envs/torch/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:775: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/pbarletta/mambaforge/envs/torch/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/pbarletta/mambaforge/envs/torch/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/home/pbarletta/mambaforge/envs/torch/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/pbarletta/mambaforge/envs/torch/lib/python3

In [6]:
idx = torch.where(voxels.view(flat_coords.shape[0]))[0]
write_pdb_dots(flat_coords[idx],
               voxels.view(flat_coords.shape[0])[idx], idx.shape[0], "../non_zero_cubo.pdb")

---------

### old stuff

In [ ]:
g = gaussian(3, std=.6)
g3d = g[None, :, None] @ g[:, None, None] @ g[None, None, :]
g3d /= np.sum(g3d)
g3d = torch.from_numpy(g3d[None, None, :, :].astype(np.float32))

xx = F.conv3d(voxels, g3d, padding=1)

In [ ]:
fig, ax = plt.subplots(1, 2)
l = 30
ax[0].imshow(voxels[0][0][l, :, :])
ax[1].imshow(xx[0][0][l, :, :])